# Lab | Random Forests
For this lab, you will be using the CSV files provided in the files_for_lab folder. These are cleaned versions of the learningSet data from the Case Study 'Healthcare for All'.

## Instructions
- Apply the Random Forests algorithm but this time only by upscaling the data.
- Use Feature Selections that you have learned in class to decide if you want to use all of the features (PCA, etc)
- Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the bussiness?

## Importing and checking if data is clean

In [15]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.feature_selection import VarianceThreshold 
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [4]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [5]:
na = pd.DataFrame(data.isna().sum(), columns=['NA_status'])
na = na[na['NA_status']!=0]
na

,NA_status


In [6]:
data.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3

## Random forest with upscaling

### X-y split

In [7]:
y = data['TARGET_B']
X = data.drop(['TARGET_B', 'TARGET_D'], axis = 1)


numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numericalX)
numerical_scaled = pd.DataFrame(numerical_scaled)

encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

X = pd.concat([numerical_scaled, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
display(X_train.shape)
display(y_train.shape)

(76329, 354)

(76329,)

### Upsampling

In [9]:
# for downsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)
# quicker way to downsample category 0:

category_0 = trainset[trainset['TARGET_B']==0]
category_1 = trainset[trainset['TARGET_B']==1]

category_1_upsampled = resample(category_1,replace=True, n_samples = len(category_0))

trainset_new = pd.concat([category_0, category_1_upsampled], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']

In [10]:
display(X_train.shape)
display(y_train.shape)

(144972, 354)

(144972,)

## Random Forests

In [40]:
%%time
import warnings
warnings.filterwarnings('ignore')
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6190781668184201
0.6157312791489807


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[11210,  6873],
       [  460,   540]], dtype=int64)

CPU times: total: 20 s
Wall time: 20.4 s


In [47]:
# %%time
# from sklearn.model_selection import cross_val_score
# clf = RandomForestClassifier(max_depth=5,
#                              min_samples_split=20,
#                              min_samples_leaf =20)
# cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
# print(np.mean(cross_val_scores))

0.6167673742403242
CPU times: total: 7min 4s
Wall time: 7min 7s


## Feature Selection

In [23]:
%%time
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

# # This drops the columns that have a variance less than this threshold
sel = sel.fit(X_train)
X_train_sel = sel.transform(X_train)
X_train_sel = pd.DataFrame(X_train_sel)
print(X_train.shape)
print(X_train_sel.shape)

(144972, 354)
(144972, 113)
CPU times: total: 703 ms
Wall time: 806 ms


In [24]:
# There are 113 columns left with a variance >= 0.02
X_train_sel.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112
0,0.624862,0.833333,1.0,0.070707,0.393939,1.000000,1.0,0.000000,0.000000,0.959596,0.020202,0.111111,0.020202,0.010101,0.979798,0.969697,0.939394,0.974333,0.932833,0.615385,0.615385,0.656566,0.353535,0.090909,0.424242,0.373737,0.989899,1.000000,1.000000,1.000000,1.000000,0.939394,0.313131,0.878788,0.000000,0.323232,0.868687,0.919192,0.959596,0.959596,0.170940,0.078341,0.111111,0.272727,0.676768,0.484848,0.525253,0.040404,0.646465,0.848485,0.828283,0.484848,0.313131,1.000000,0.0,0.070707,0.393939,0.505051,0.909091,0.161616,0.634615,0.010101,0.101010,0.404040,0.545455,0.464646,0.515152,0.373737,1.000000,0.000000,1.000000,1.000000,0.6,0.150000,0.484784,1.0,0.666667,0.081967,0.000000,1.0,0.000000,0.857143,0.000000,0.090909,0.545455,0.090909,0.5,0.090909,0.545455,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.546392,0.666667,1.0,0.454545,0.171717,1.000000,1.0,0.000000,0.000000,0.929293,0.030303,0.424242,0.757576,0.737374,0.252525,0.121212,0.040404,0.049667,0.052333,0.153846,0.153846,0.646465,0.363636,0.000000,0.656566,0.464646,0.000000,0.010101,0.010101,0.010101,0.040404,0.000000,0.555556,0.000000,0.000000,0.030303,0.000000,0.040404,0.343434,0.686869,0.824786,0.600614,0.191919,0.292929,0.353535,0.464646,0.545455,0.000000,0.383838,0.747475,0.676768,0.767677,0.373737,0.727273,0.0,0.454545,0.171717,0.646465,0.919192,0.535354,0.980769,0.000000,0.000000,0.000000,0.040404,0.969697,0.979798,0.000000,1.000000,0.000000,0.959596,0.190476,0.4,0.316667,0.395426,0.0,1.000000,0.557377,0.557692,1.0,0.666667,0.642857,0.453608,0.000000,0.090909,0.181818,0.5,0.181818,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.340206,0.833333,1.0,0.000000,0.000000,1.000000,1.0,0.000000,0.000000,0.919192,0.010101,0.252525,0.313131,0.242424,0.686869,0.686869,0.656566,0.153333,0.167167,0.384615,0.384615,0.282828,0.727273,0.020202,0.474747,0.383838,0.010101,0.050505,0.343434,0.838384,0.989899,0.010101,0.272727,0.515152,0.000000,0.626263,0.575758,0.939394,1.000000,1.000000,0.688034,0.357911,0.121212,0.121212,0.494949,0.767677,0.242424,0.464646,0.686869,0.848485,0.797980,0.787879,0.686869,0.575758,1.0,0.393939,0.080808,0.303030,0.898990,0.565657,0.192308,0.131313,0.313131,0.959596,0.959596,0.060606,0.262626,0.717172,1.000000,0.000000,1.000000,0.476190,0.0,0.250000,0.909489,0.0,0.666667,0.524590,0.750000,0.5,0.333333,0.785714,0.659794,0.000000,0.000000,0.545455,0.5,0.000000,0.818182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.624862,0.666667,1.0,0.262626,0.515152,0.444444,0.0,1.000000,0.000000,0.868687,0.080808,0.333333,0.878788,0.868687,0.111111,0.060606,0.020202,0.086833,0.099500,0.230769,0.153846,0.747475,0.262626,0.010101,0.696970,0.575758,0.010101,0.020202,0.111111,0.252525,0.535354,0.000000,0.525253,0.010101,0.010101,0.020202,0.030303,0.101010,0.383838,0.666667,0.000000,0.413210,0.303030,0.383838,0.474747,0.424242,0.585859,0.070707,0.212121,0.676768,0.616162,0.666667,0.575758,0.707071,1.0,0.262626,0.515152,0.707071,0.979798,0.555556,0.538462,0.040404,0.111111,0.303030,0.555556,0.454545,0.848485,0.141414,1.000000,0.000000,0.989899,0.428571,0.4,0.366667,0.600137,0.0,0.333333,0.836066,0.692308,1.0,0.333333,0.571429,0.000000,0.090909,0.181818,0.454545,0.0,0.454545,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.624862,0.666667,1.0,0.505051,0.232323,0.111111,0.0,0.969697,0.040404,0.949495,0.010101,0.454545,0.878788,0.858586,0.101010,0.040404,0.000000,0.095667,0.103500,0.230769,0.230769,0.686869,0.323232,0.

In [25]:
y_train

22182    1
8739     1
15536    0
62196    1
31749    1
        ..
22904    0
86383    0
9229     0
53049    1
35800    1
Name: TARGET_B, Length: 144972, dtype: int64

In [27]:
%%time

X_test_sel = sel.transform(X_test)
X_test_sel = pd.DataFrame(X_test_sel)
print(X_test.shape)
print(X_test_sel.shape)

(19083, 354)
(19083, 113)
CPU times: total: 15.6 ms
Wall time: 18.8 ms


## Random Forest with selected features

In [28]:
%%time
import warnings
warnings.filterwarnings('ignore')
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)

clf.fit(X_train_sel, y_train)
print(clf.score(X_train_sel, y_train))
print(clf.score(X_test_sel, y_test))

y_pred = clf.predict(X_test_sel)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6229271859393538
0.586018969763664


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[10629,  7454],
       [  446,   554]], dtype=int64)

CPU times: total: 7.91 s
Wall time: 8.26 s


In [31]:
donations = data[data['TARGET_B']==1]
donations['TARGET_D'].mean()

15.624344414619037

The feature selection slitghtly improved the corect prediction of donators but decreased the accuracy of not donors.
Not using the feature selection would mean losing donators, but using it would mean more money spent on mailing the kits.

Based on the mean donations, the extra cost for mailing the kits would still be lower than the additional donations expected, therefore I would still use the feature selection and mail the donation kits to more people.